# Peel Results Parser

This Jupyter notebook parses the *results* folder of a *peel*-bundle.

In [22]:
import numpy as np
import pandas as pd

import os
import itertools as it
import json
import re
import warnings
import glob

## Parse Runtimes

In [23]:
def parse_runtimes(suite):
    runtimes = []
    
    for run_path in glob.iglob(os.path.join(suite, '*.top*.run*')):
        scale_out, run = int(run_path[-9:-6]), int(run_path[-2:])
        state_file = os.path.join(run_path, 'state.json')
        with open(state_file) as f:
            j = json.load(f)
            if j['runExitCode'] != 0:
                warnings.warn('%s did not complete successfully.' % run_path)
            else:
                runtimes.append( (scale_out, j['runTime']) )
    
    return pd.DataFrame(runtimes, columns=['Nodes', 'Runtime'])

In [26]:
# pagerank on spark
#runtimes = parse_runtimes('results/pageRank.spark.scaleout/')
#runtimes.to_csv('pagerank.spark.csv', index=False)

# tpc-h query 10 on flink
runtimes = parse_runtimes('results/suite.flink.tpchq10.scaleout/')
runtimes.to_csv('tpchq10.flink.csv', index=False)

# kmeans on flink
#runtimes = parse_runtimes('results/suite.flink.kmeans.scaleout/')
#runtimes.to_csv('kmeans.flink.csv', index=False)

## Parse Dstat Statistics (TODO)

In [4]:
def get_suites(results_path):
    runre = re.compile(r'\.run\d\d$')

    suites_dict = {}
    for suite_path in glob.iglob('%s*/' % os.path.join(results_path, '')):        
        experiments = map(lambda p: os.path.split(p)[1][:-6], glob.iglob('%s*.run*' % suite_path))
        suites_dict[suite] = experiments
    
    return suites_dict

In [5]:
results_path = 'results/'
master = 'wally001'
suites = get_suites(results_path)
print suites

NameError: global name 'suite' is not defined

In [6]:
def get_dstat_statistics(run_path):
    dstat_logs_folder = os.path.join(run_path, 'logs', 'dstat', 'dstat-0.7.2')
    dstat_logs = os.listdir(dstat_logs_folder)
    dstat_logs = filter(lambda x: not x.endswith(master + '.cit.tu-berlin.de.csv'), dstat_logs)

    frames = []
    data = []
    for dstat_log in dstat_logs:
        log_file = os.path.join(dstat_logs_folder, dstat_log)
        usecols = ['usr','sys','idl','wai','hiq','siq','recv','send','read','writ']
        df = pd.read_csv(log_file, skiprows=6, header=0, usecols=usecols)
        frames.append(df)
        #X = df.values
        #data.append(X)
        
    #n = np.min([X.shape[0] for X in data])
    #data = np.dstack([X[:n,:] for X in data])
    #data = np.transpose(data, axes=(2,0,1))
    #return data
    
    keys = map(lambda s: s[-12:-4], dstat_logs)
    df = pd.concat(frames, keys=keys)
    print df.index.
    print df.ix['wally060'].head()
    print df.ix['wally060'].values

    

_ = get_dstat_statistics('./results/tpchq10.cluster/tpchq10.flink.top004.run01/', cols=range(1,7)+range(53,57))

SyntaxError: invalid syntax (<ipython-input-6-c58bfa7d0d73>, line 23)

In [7]:
def parse(name, suite, num_runs=5):
    d = OrderedDict()
    for experiment in suites[suite]:
        path = os.path.join(results_path, suite, experiment)
        
        runtimes = []
        for i in range(num_runs):
            run_path = '%s.run%02d' % (path, i+1)
            state = os.path.join(run_path, 'state.json')
            with open(state) as f:
                j = json.load(f)
                runtime = j['runTime']
        
            dstats = get_dstat_statistics(run_path)
            key = 'top%03d.run%02d' % (dstats.shape[0], i+1)
            #scaleout_key = 'top%03d' % (dstats.shape[0],)
            #d.setdefault(scaleout_key, OrderedDict())
            #run_key = 'run%02d' % (i+1,)
            #d[scaleout_key][run_key] = OrderedDict()
            #d[scaleout_key][run_key]['dstat'] = dstats
            
            d[key] = dstats
            
            #d[scaleout_key][run_key]['runtime'] = runtime

    np.savez_compressed(name+'.npz', **d)
    
#parse('tpchq3', 'tpchq3.cluster')
parse('tpchq10', 'tpchq10.cluster')
#parse('tpchq10', 'tpchq10.cluster')

NameError: global name 'OrderedDict' is not defined